In [8]:
import pm4py
import pandas as pd
import networkx as nx

ocel = pm4py.read_ocel2_sqlite("/home/grkmr/Downloads/age_of_empires_ocel2.sqlite")

def compute_event_object_graph(ocel: pm4py.OCEL) -> nx.DiGraph:
    """
    Builds a directed event-object graph (EOG) from an OCEL using directly-follows per object.

    Each event becomes a node, and edges are added between events that refer to the same object
    and occur consecutively in time.

    :param ocel: PM4Py OCEL object
    :return: networkx.DiGraph representing the EOG
    """
    event_object_graph = nx.DiGraph()

    # Step 1: Add each event as a node in the graph
    event_ids = ocel.events[ocel.event_id_column].to_list()
    event_object_graph.add_nodes_from(event_ids)

    # Step 2: Sort and group by object, then generate adjacent event pairs
    object_with_sorted_events = (
        ocel.relations
        .sort_values(ocel.event_timestamp)
        .groupby(ocel.object_id_column)["ocel:eid"]
        .apply(lambda lst: list(zip(lst, lst[1:])))  # adjacent pairs only
        .explode()
        .dropna()
        .to_list()
    )

    # Step 3: Add those event-event edges to the graph
    event_object_graph.add_edges_from(object_with_sorted_events)

    return event_object_graph

def compute_process_executions_connected_components(ocel:pm4py.OCEL):
    return sorted(
            nx.weakly_connected_components(compute_event_object_graph(ocel)), key=len, reverse=True
        )


/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/ocel/importer/sqlite/variants/ocel20.py:75: UserWarning: There are unsatisfied OCEL 2.0 constraints in the given relational database: ['const_14_primary_key_object_event_map_type_tables', 'const_15_primary_key_object_event_tables', 'const_16_primary_key_event_object_table', 'const_17_primary_key_object_object_table', 'const_18_primary_key_event_type_spec_tables', 'const_19_foreign_key_event', 'const_20_foreign_key_object', 'const_21_foreign_key_event_object', 'const_22_foreign_key_object_object', 'const_23_foreign_key_event_type_specific', 'const_24_foreign_key_object_type_specific']
  warnings.warn("There are unsatisfied OCEL 2.0 constraints in the given relational database: "+str(unsatisfied))
/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/ocel/util/ocel_consistency.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [4]:
from itertools import combinations

In [10]:
pm4py.ocel_get_object_types(ocel)

['Archery Range',
 'Barracks',
 'Blacksmith',
 'Bombard Tower',
 'Castle',
 'Dock',
 'Donjon',
 'Farm',
 'Feitoria',
 'Fish Trap',
 'Gate',
 'House',
 'Krepost',
 'Lumber Camp',
 'Market',
 'Match',
 'Mill',
 'Mining Camp',
 'Monastery',
 'None',
 'Outpost',
 'Palisade Gate',
 'Player',
 'Session',
 'Siege Workshop',
 'Stable',
 'Town Center',
 'University',
 'Villager',
 'Watch Tower']

In [24]:
def leading_type_process_executions(ocel:pm4py.OCEL, leading_type :str):
    objects = ocel.relations.groupby(ocel.object_id_column).agg({ocel.event_id_column:list,ocel.object_type_column :'first'}).to_dict("index").items()
    edges = ocel.relations.groupby(ocel.event_id_column)[ocel.object_id_column].apply(lambda x: list(combinations(x, 2))).explode().dropna().to_list()

    object_graph = nx.Graph()
    object_graph.add_nodes_from(objects)
    object_graph.add_edges_from(edges)

    
    cases = []

    for object_id in ocel.objects[ocel.objects[ocel.object_type_column] == leading_type][ocel.object_id_column]:
        print(object_id)
        relevant_objects = {object_id}
        events = set(object_graph.nodes[object_id][ocel.event_id_column])
        next_level_objects = list(object_graph.neighbors(object_id))  
        visited_object_types = {leading_type: 0}
    
        for level in range(1, len(pm4py.ocel_get_object_types(ocel))):
            to_be_next_level_objects = []
    
            for current_object_id in next_level_objects:
                current_object_type = object_graph.nodes[current_object_id][ocel.object_type_column] 
    
                if current_object_type not in visited_object_types:
                    visited_object_types[current_object_type] = level
                elif visited_object_types[current_object_type] != level:
                    continue
    
                to_be_next_level_objects.extend(object_graph.neighbors(current_object_id))
                relevant_objects.add(current_object_id)
                    
            next_level_objects = list(set(to_be_next_level_objects)) 
    
        cases.append((events, relevant_objects))
    return cases

In [ ]:
from itertools import combinations
from collections import defaultdict

event_groups = defaultdict(list)
for _, row in ocel.relations[[ocel.event_id_column, ocel.object_id_column]].itertuples(index=False):
    event_groups[row[0]].append(row[1])

# Now generate combinations efficiently
edges = []
for obj_ids in event_groups.values():
    if len(obj_ids) > 1:
        edges.extend(combinations(obj_ids, 2))
